In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import itertools
import re
from operator import itemgetter

nlp = en_core_web_sm.load()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

['hillary-clinton-emails', 'englishnames3']


# Named Entity Recognition 

*Daan van Kooten 10-2018
*

**This Kernel consists of:**
1. A first try to find all NE's in an examplestring from the Clinton Mails using spacy.
2. Replace all names and organisation in the body text of the e-mails by 'Name' or 'Organisation'

In [3]:
data_mail = pd.read_csv('../input/hillary-clinton-emails/Emails.csv', header=0)
data_mail.head(5)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,ExtractedSubject,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,FW: Wow,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,HRC_Email_296,Re: Chris Stevens,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,HRC_Email_296,FVV: Cairo Condemnation - Final,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


## 1. A first try to find all NE's in an examplestring from the Clinton Mails.

Example sentence from clinton mails:

In [4]:
BodyText = data_mail.ExtractedBodyText
ex = BodyText[1398]
print(ex)


Dear Secretary Clinton,
I am seeing Michelle on Tuesday at 3 pm. Thank you very much for making this happen, and I will let you know how the
meeting goes.
Congratulations on all the glowing first 100 days reviews. Hope your testimony yesterday went well. Mine is attached,
fyi.
All best, and thank you again, Jackie
Jacqueline Newmyer
President, Long Term Strategy Group
12 Eliot St., Cambridge, MA 02138
B6
617-661-1626 (fax)
www.ltstrategy.com


Apply word tokenization and part-of-speech tagging to the sentence.



In [8]:
doc = nlp(ex)
doc

Dear Secretary Clinton,
I am seeing Michelle on Tuesday at 3 pm. Thank you very much for making this happen, and I will let you know how the
meeting goes.
Congratulations on all the glowing first 100 days reviews. Hope your testimony yesterday went well. Mine is attached,
fyi.
All best, and thank you again, Jackie
Jacqueline Newmyer
President, Long Term Strategy Group
12 Eliot St., Cambridge, MA 02138
B6
617-661-1626 (fax)
www.ltstrategy.com

In [9]:
print([(X.text, X.label_) for X in doc.ents])  

[('Clinton', 'PERSON'), ('Michelle', 'PERSON'), ('Tuesday', 'DATE'), ('3 pm', 'TIME'), ('first 100 days', 'DATE'), ('yesterday', 'DATE'), ('Jackie', 'PERSON'), ('Jacqueline Newmyer', 'PERSON'), ('Long Term Strategy Group', 'ORG'), ('12', 'CARDINAL'), ('Cambridge', 'GPE'), ('02138', 'CARDINAL')]


In [10]:
displacy.render(nlp(ex), jupyter=True, style='ent')

In [11]:
displacy.render(nlp(ex), style='dep', jupyter = True, options = {'distance': 120})

## 2.  Remove NER in dataset:

- Remove names and organisations based on Spacy NER
- Remove names based on large list of common English names
- Remove all mailaddresses

In [14]:
def dicReplace(words, replacements, string):
    #string = ex
    #words = entityArray
    #replacements = mailMatch
    if words.size > 0:
        dic_NER = dict(zip(words, replacements))
        rep = dic_NER
        # use these three lines to do the replacement
        rep = dict((re.escape(k), v) for k, v in rep.items())
        pattern = re.compile("|".join(rep.keys()))
        string = pattern.sub(lambda m: rep[re.escape(m.group(0))], string)
        transformedString = pattern.sub(lambda m: rep[re.escape(m.group(0))], string)
    else: transformedString = string
    return(transformedString)

In [15]:
def NERTransformer(string):
    ent_list = [(X.text, X.label_) for X in nlp(ex).ents]  
    entityArray= np.array([item[1] for item in  ent_list]) 
    wordArray = np.array([item[0] for item in  ent_list])
    poIndex = [(entityArray == 'ORG') | (entityArray == 'PERSON')]
    entityArray2 = entityArray[tuple(poIndex)]
    wordArray2 = wordArray[tuple(poIndex)]
    transformedString= dicReplace(wordArray2, entityArray2, string)
    return(transformedString)

In [17]:
data_names= pd.read_csv('../input/englishnames3/englishNames.csv', header=0)
data_names['Name'] = ' ' + data_names['Name'].astype(str) + ' ' #add spaces 
data_names['Entity'] = ' ' + data_names['Entity'].astype(str) + ' ' #add spaces 
data_names

,Name,Entity
0,Aabha,PERSON
1,Aadam,PERSON
2,Aaden,PERSON
3,Aadhan,PERSON
4,Aadhith,PERSON
5,Aadhrith,PERSON
6,Aadhya,PERSON
7,Aadhyaa,PERSON
8,Aadi,PERSON
9,Aadia,PERSON


In [19]:
def NamesTransformer(string):
    transformedString = dicReplace(data_names['Name'], data_names['Entity'], string)
    return(transformedString)

In [20]:
def EmailTransformer(string):
    mailMatch = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', string)
    mailMatch = np.array(mailMatch)
    entityArray = np.repeat('EMAILADDRESS', len(mailMatch))
    transformedString = dicReplace(mailMatch,entityArray, string)
    return(transformedString)

In [21]:
def anonymous(string):
    stringAno1 = NERTransformer(string)
    stringAno2 = NamesTransformer(stringAno1)
    stringAno3 = EmailTransformer(stringAno2)
    return(stringAno3)

In [22]:
anonymous(ex)

'Dear Secretary PERSON,\nI am seeing PERSON on Tuesday at 3 pm. Thank you very much for making this happen, and I will let you know how the\nmeeting goes.\nCongratulations on all the glowing first 100 days reviews. PERSON your testimony yesterday went well. Mine is attached,\nfyi.\nAll best, and thank you again, PERSON\nPERSON\nPresident, ORG\n12 PERSON St., Cambridge, MA 02138\nB6\n617-661-1626 (fax)\nwww.ltstrategy.com'

In [ ]:
#BodyText2 = BodyText[~BodyText.isnull()] #remove nans
#anonymous(BodyText2[2556])
#BodyText2.apply(anonymous)